# A multilayer fitting model

One of the main tools in `EasyReflectometry` is the [assemblies library](../sample/assemblies_library.rst). 
This allows the user to define their model, using specific parameters for their system of interest (if it is included in the assemblies library). 
These assemblies will impose necessary constraints and computational efficiencies based on the assembly that is used. 

In this tutorial, we will look at one of these assemblies, that of a `RepeatingMultilayer` ([documented here](../sample/assemblies_library.rst#repeatingmultilayer)). 
This tutorial is based on an example from the [BornAgain](https://www.bornagainproject.org) documentation looking at [specular reflectivity analysis](https://www.bornagainproject.org/m/py/fitting/extended/fit-specular-data/). 
Before performing analysis, we should import the packages that we need.

First configure matplotlib to place figures in notebook and import needed modules

In [ ]:
%matplotlib inline

import numpy as np
import scipp as sc
import EasyReflectometry
import refl1d
from EasyReflectometry.data import load
from EasyReflectometry.sample import Layer
from EasyReflectometry.sample import Sample
from EasyReflectometry.sample import Material
from EasyReflectometry.sample import RepeatingMultilayer
from EasyReflectometry.experiment.model import Model
from EasyReflectometry.calculators import CalculatorFactory
from EasyReflectometry.fitting import Fitter
from EasyReflectometry.plot import plot

As mentioned in the [previous tutorial](./simple_fitting.rst), we share the version of the software packages we will use. 

In [ ]:
print(f'numpy: {np.__version__}')
print(f'scipp: {sc.__version__}')
print(f'EasyReflectometry: {EasyReflectometry.__version__}')
print(f'Refl1D: {refl1d.__version__}')

## Reading in experimental data

The data that we will investigate in this tutorial was generated with [GenX](https://aglavic.github.io/genx/) and is stored in an `.ort` [format file](https://github.com/reflectivity/file_format/blob/master/specification.md). 

In [ ]:
data = load('_static/repeating_layers.ort')
data

This data is very featureful, with many fringes present (arising from the multilayer structure)

In [ ]:
plot(data)

## Building our model

The system that was used to produce the data shown above is based on a silicon subphase, with a repeating multilayer of nickel and titanium grown upon it. 
Typcially, under experimental conditions, the producer of the sample will know how many repeats there will be of the multilayer system (as these are grown using some vapour disposition or sputtering method that the producer controls). 
We show the model that will be used graphically below. 

<center>
    <img src='_static/repeating.png' alt='A slab model description of the repeating multilayer system.' width='300px'></img>
</center>
<center>
    A slab model description of the repeating multilayer, showing the four layers of vacuum, titanium, nickel and silicon, with the titanium/nickel layers being repeated 10 times.
</center>

To construct such a layer structure, first we create each of the materials and associated layers

In [ ]:
vacuum = Material.from_pars(0, 0, 'Vacuum')
ti = Material.from_pars(-1.9493, 0, 'Ti')
ni = Material.from_pars(9.4245, 0, 'Ni')
si = Material.from_pars(2.0704, 0, 'Si')

In [ ]:
superphase = Layer.from_pars(vacuum, 0, 0, 'Vacuum Superphase')
ti_layer = Layer.from_pars(ti, 40, 0, 'Ti Layer')
ni_layer = Layer.from_pars(ni, 70, 0, 'Ni Layer')
subphase = Layer.from_pars(si, 0, 0, 'Si Subphase')

Then, to produce the repeating multilayer, we use the `RepeatingMultilayer` [assembly type](../sample/assemblies_library.rst#RepeatingMultilayer). 
This can be constructed in a range of different ways, however here we pass a list of `Layer` type objects and a number of repetitions. 

In [ ]:
rep_multilayer = RepeatingMultilayer.from_pars([ti_layer, ni_layer], repetitions=10, name='NiTi Multilayer')
rep_multilayer

From these objects, we can construct our structure and combine this with a scaling, background and resolution (since this data is simulated there is no background or resolution smearing). 

In [ ]:
sample = Sample.from_pars(superphase, rep_multilayer, subphase, name='Multilayer Structure')
model = Model.from_pars(sample, 1, 0, 0, 'Multilayer Model')

In the analysis, we will only vary a single parameter, the thickness of titanium layer. 

In [ ]:
ti_layer.thickness.bounds = (10, 60)

## Choosing our calculation engine

In the [previous tutorial](./simple_fitting.rst), we used the default [refnx](https://refnx.readthedocs.io/) engine for our analysis. 
Here, we will change our engine to be [*Refl1D*](https://refl1d.readthedocs.io/en/latest/).
This is achieved with the `interface.switch('refl1d')` method below. 

In [ ]:
interface = CalculatorFactory()
interface.switch('refl1d')
model.interface = interface
print(interface.current_interface.name)

## Performing an optimisation

The [easyScience](https://easyscience.github.io/easyCore/) framework allows us to access a broad range of optimisation methods.
Below, we have selected the [differential evolution method from lmfit](https://lmfit.github.io/lmfit-py/examples/example_diffev.rst).

In [ ]:
fitter = Fitter(model)
analysed = fitter.fit(data, method='differential_evolution')
analysed

We can visualise the analysed model and SLD profile with the `plot` function. 

In [ ]:
plot(analysed)

The value of the titanium layer thickness that gives this best fit can be found from the relavant object.
Note that the uncertainty of 0 is due to the use of the lmfit differential evolution algorithm, which does not include uncertainty analysis. 

In [ ]:
ti_layer.thickness

This result of a thickness of 30 Å is the same as that which is used to produce the data. 